In [ ]:
import aiohttp
import asyncio
import os

from tenacity import retry, stop_after_attempt
from time import perf_counter

In [99]:
@retry(stop=stop_after_attempt(1))
async def fetch_products(url: str):
    start = perf_counter()
    path=url.split("v1")[1]
    print(f'{start}:\t{path}')

    params = {
        'apiKey': os.environ.get('API_KEY_BESTBUY'),
        'pageSize': 10,
        'page': 1, 
        'format': 'json',
        'show': 'all',
        'sort': 'salePrice.asc'
    }

    async with aiohttp.ClientSession() as session:
        async with session.get(url, params=params) as response:
            data = await response.json()
            print(f'{path}... took {perf_counter() - start:.2f}s')
            return data

In [100]:
urls = [
    "https://api.bestbuy.com/v1/products(productTemplate in(Televisions)&screenSizeClassIn>39&salePrice<999)",
    "https://api.bestbuy.com/v1/products(categoryPath.name=laptop*&onSale=true&details.value=nvidia)"
]

results = await asyncio.gather(*[fetch_products(url) for url in urls])
print(f'Found {sum(r["total"] for r in results)} total products')

2857743.306845:	/products(productTemplate in(Televisions)&screenSizeClassIn>39&salePrice<999)
2857743.3074585:	/products(categoryPath.name=laptop*&onSale=true&details.value=nvidia)
/products(productTemplate in(Televisions)&screenSizeClassIn>39&salePrice<999)... took 0.69s
/products(categoryPath.name=laptop*&onSale=true&details.value=nvidia)... took 0.86s
Found 280 total products
